<a href="https://colab.research.google.com/github/dotsnangles/Retrieval-Based-Chatbot/blob/main/Data_Analysis_Poly_Encoder_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets folium==0.2.1

     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 362 kB 51.4 MB/s 
     |████████████████████████████████| 69 kB 5.4 MB/s 
     |████████████████████████████████| 101 kB 3.2 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
     |████████████████████████████████| 6.6 MB 39.9 MB/s 
     |████████████████████████████████| 1.1 MB 65.0 MB/s 
     |████████████████████████████████| 140 kB 79.0 MB/s 
     |████████████████████████████████| 212 kB 48.8 MB/s 
     |████████████████████████████████| 127 kB 61.3 MB/s 
     |████████████████████████████████| 271 kB 59.5 MB/s 
     |████████████████████████████████| 144 kB 60.0 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 


In [2]:
!git clone -q https://github.com/dotsnangles/Poly-Encoder.git

In [3]:
!gdown -q --folder 1uC1pSCrh9xlieF60z78QuCg7OxvU9kUs
!mv /content/dstc7/*.json /content/Poly-Encoder/dstc7
!mv /content/dstc7/*.tsv /content/Poly-Encoder/dstc7

In [4]:
%cd /content/Poly-Encoder/dstc7
!bash parse.sh
%cd /content

/content/Poly-Encoder/dstc7
tcmalloc: large alloc 1859141632 bytes == 0x2ca8000 @  0x7f457d4101e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x5939af 0x594cd3 0x594f8e 0x59526e 0x5bfba0 0x59aeca 0x515655 0x549e0e 0x593fce 0x548ae9 0x51566f 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7f457d00dc87 0x5b621a
tcmalloc: large alloc 1859141632 bytes == 0x719ac000 @  0x7f457d4101e7 0x4a3940 0x52ab72 0x527cf3 0x51d358 0x59358d 0x548c51 0x51566f 0x549576 0x4bcb19 0x59c019 0x59588e 0x595e64 0x4d8924 0x5bfbcb 0x59aeca 0x515655 0x549e0e 0x593fce 0x548ae9 0x51566f 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7f457d00dc87 0x5b621a
/content


In [5]:
# with open('/content/Poly-Encoder/dstc7/train.txt', 'r') as f:
#     lines = f.readline()
# print(lines)
# # idx = 0
# # for line in lines:
# #     print(line)
# #     idx += 1
# #     if idx == 100:
# #         break

In [6]:
file_path = '/content/Poly-Encoder/dstc7/train.txt'

In [7]:
# with open(file_path, encoding='utf-8') as f:
#     for i, line in enumerate(f):
#         if i == 30:
#             break
#         print(line)

In [8]:
train_data_source = []
train_neg_responses = []
sample_cnt = None

In [9]:
with open(file_path, encoding='utf-8') as f:
    group = {
        'context': None,
        'responses': [],
        'labels': []
    }
    for line in f:
        split = line.strip('\n').split('\t')
        lbl, context, response = int(split[0]), split[1:-1], split[-1]
        if lbl == 1 and len(group['responses']) > 0:
            train_data_source.append(group)
            group = {
                'context': None,
                'responses': [],
                'labels': []
            }
            if sample_cnt is not None and len(train_data_source) >= sample_cnt:
                break
        else:
                train_neg_responses.append(response)
        group['responses'].append(response)
        group['labels'].append(lbl)
        group['context'] = context
    if len(group['responses']) > 0:
        train_data_source.append(group)

In [10]:
len(train_data_source)

100000

In [28]:
idx = 0

In [29]:
train_data_source[idx]['context']

['participant 1: Hi, I want to run a graphical application from the command line, here is the script I wrote: https://paste8.com/4XQiHrXZ - it\'s Ubuntu Server 12.04 + Unity. What I get is an error from xhost "unable to open display :0" and from the graphical application I want to use (Sikuli) "Can\'t connect to X11 window server using \':0\' as the value of the DISPLAY variable.". I\'ve tried using DISPLAY:=1 as I use this number when connecting with a VNC client but it doesn\'t wo. rk either...',
 'participant 2: is X running?',
 'participant 1: I think so: https://paste8.com/HvhlT6vO.  maybe you prefer this check: https://paste8.com/0OhcBmfB',
 'participant 2: these days you use lightdm (or gdm, kdm) to start X, so i\'d try to kill what\'s there but unreachable with "sudo killall X" and then "sudo service lightdm start"',
 'participant 1: what do I kill precisely? because it says "X: no such process"',
 'participant 2: nothing, X wasn\'t running if you get that, proceed with "sudo s

In [30]:
train_data_source[idx]['responses']

["are you still running in root terminal? if so, try from a non-root one.. i've got to go prepare dinner so i'll be afk .. ask the channel for help on that, hope you get things up",
 'quite',
 "hey join #.  so i can tell you things that the kiddies won't latch on to",
 'did you complete the ubuntu installation??',
 'how did you run mplayer?',
 "a skype replacement? i think google talk works in the US (i wouldn't know i'm in uk)",
 "you only need the libraries for each.... ... which isn't too expensive",
 'try sudo apt-get install ubuntu-desktop',
 "which error is that?. btw don't make us ask these questions",
 "I'm not sure there is one, you may need to just tweak ~/.config/xfce4.",
 "- you'll figure it out - best to you.",
 'try running "sudo updatedb" then "locate eclipse". youll find it then',
 "I don't remember though. anyone else? how to get verbose output during a boot? turn of boot splash during live cd?",
 "It's painful that's true :p.",
 'Try cat /etc/issue.  Sorry, debian_ver

In [31]:
train_data_source[idx]['labels']

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
train_neg_responses[0]

"are you still running in root terminal? if so, try from a non-root one.. i've got to go prepare dinner so i'll be afk .. ask the channel for help on that, hope you get things up"

In [32]:
file_path = '/content/Poly-Encoder/dstc7/dev.txt'

In [33]:
dev_data_source = []
dev_neg_responses = []
sample_cnt = 3

In [34]:
with open(file_path, encoding='utf-8') as f:
    group = {
        'context': None,
        'responses': [],
        'labels': []
    }
    for line in f:
        split = line.strip('\n').split('\t')
        lbl, context, response = int(split[0]), split[1:-1], split[-1]
        if lbl == 1 and len(group['responses']) > 0:
            dev_data_source.append(group)
            group = {
                'context': None,
                'responses': [],
                'labels': []
            }
            if sample_cnt is not None and len(dev_data_source) >= sample_cnt:
                break
        else:
                dev_neg_responses.append(response)
        group['responses'].append(response)
        group['labels'].append(lbl)
        group['context'] = context
    if len(group['responses']) > 0:
        dev_data_source.append(group)

In [35]:
idx = 0

In [36]:
dev_data_source[idx]['context']

['participant 1: hey guys, does your livecd have chroot installed? and bash?',
 'participant 2: sure',
 'participant 1: does it have everything I need to format a partition ext2?. and ext3?',
 'participant 2: yep',
 'participant 1: yay I can use it to install gentoo. !',
 'participant 2: lol. LOL',
 'participant 1: =-). brb rebooting into ubuntu',
 'participant 2: form last week:. 04:21:47] <findme> this is a big crowd here. [04:21:53] <findme> have all gentoo users moved here ?',
 'participant 1: to bad its still using apt I would switch in a heart beat if it had its own package manager']

In [37]:
dev_data_source[idx]['responses']

['what are you missing in apt ?',
 "issues with msn?. I'm experiencing them on windows atm, current msn version",
 "<> AmaroqWolf: alias='sudo admincommand'.  <AmaroqWolf>  aw, can't make myself type sudo? I like it better that way.",
 'lspci will list your hardware, take a look at the VGA line',
 'even if you set gnome proxy, not all applications obey it. if your particular application has a network settings option, set the proxy there.',
 'beryll had that 3d cube effect.',
 'Change both Sound In and Sound Out on the sound devices option to: HDA Intel (hw:intel,0).  Then do a test call',
 "does setting the cpu frequency with the gnome panel applet work for you? It works for me....  that might be _because_ I don't have (again, to the best of my knowledge) any cpu-frequency-setting app running",
 'export http_proxy=http://proxy:port export ftp_proxy=http://proxy:port',
 'what laptop?',
 'for what?',
 "atleast you're now using the fglrx drivers. anyways, check around the bug track and ty

In [38]:
dev_data_source[idx]['labels']

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [23]:
dev_neg_responses[100]

"maybe it works under your account because of your envvironmental variables, eg. current directory etc which aren't set during boot"